In [ ]:
import pandas as pd 
import ipywidgets as widgets
import datetime
import sqlalchemy
import psycopg2
import golf_functions

This notebook is used to enter info on each hole for a course/tee. This will be entered into 'hole_info'. 

You will need the following info:
- course/tee
- number of holes
- for each hole
    - par
    - distance
    - course handicap

In [ ]:
course_select_query = '''
Select ci.course_id
    ,Max(ci.course_version_id) as course_version_id
    ,cn.course_name
    ,ci.tee
from "Courses"."course_info" ci 
left join "Courses"."course_name" cn on cast(ci.course_id as int) = cn.course_id
group by ci.course_id, ci.tee, cn.course_name
'''
course_tee_select = golf_functions.pull_from_sql(course_select_query)
course_tee_select['course_tee'] = course_tee_select['course_name'] + ' - ' + course_tee_select['tee']

In [ ]:
# selecting the course/tee and number of holes
ct_select = widgets.Dropdown(options = list(course_tee_select['course_tee']))
ct_select

In [ ]:
course_info = course_tee_select[course_tee_select['course_tee'] == ct_select.value]
course_info.reset_index(inplace=True)

In [ ]:
num_holes = widgets.IntText()
num_holes

In [ ]:
# starting df used in load_sql
# only needed columns will be used later
holes = pd.DataFrame()
holes['hole'] = range(1,num_holes.value + 1)
holes['course_id'] = course_info['course_id'][0]
holes['course_version_id'] = course_info['course_version_id'][0]
holes['course_name'] = course_info['course_name'][0]
holes['tee'] = course_info['tee'][0]
holes['hole']= holes['hole'].astype(str)
holes

In [ ]:
# creating labels and widgets used in entry. 
# globals() is not always the best way to creat variabls is useful in this situation is creating variabls with a four loop. The four loop reduced the lines of code needed in the notebook. 
for h in range(1,num_holes.value + 1):
    globals()['par_%s' % h + '_lbl'] = widgets.Label('par')
    globals()['par_%s' % h + '_ent'] = widgets.Text()
    globals()['distance_%s' % h + '_lbl'] = widgets.Label('distance')
    globals()['distance_%s' % h + '_ent'] = widgets.IntText()
    globals()['handicap_%s' % h + '_lbl'] = widgets.Label('Course Handicap')
    globals()['handicap_%s' % h + '_ent'] = widgets.Text()
    display(widgets.Label('For Hole - ' + str(h)))
    display(globals()['par_%s' % h + '_lbl'])
    display(globals()['par_%s' % h + '_ent'])
    display(globals()['distance_%s' % h + '_lbl'])
    display(globals()['distance_%s' % h + '_ent'])
    display(globals()['handicap_%s' % h + '_lbl'])
    display(globals()['handicap_%s' % h + '_ent'])

In [ ]:
# creating listed from widgets entered info
hole_list = []
par_list = []
distance_list = []
handicap_list = []
for h in range(1,num_holes.value + 1):
    hole_list.append(str(h))
    par_list.append(globals().get('par_%s' % h + '_ent').value)
    distance_list.append(globals().get('distance_%s' % h + '_ent').value)
    handicap_list.append(globals().get('handicap_%s' % h + '_ent').value)
# creating df needed for load_sql
hole_info_dict = {
    'hole':hole_list,
    'par':par_list,
    'distance':distance_list,
    'handicap':handicap_list
}

In [ ]:
# merging new df and df made above. Selecting info needed for hole_info table
hole_info = pd.merge(holes,pd.DataFrame(hole_info_dict,columns=hole_info_dict.keys()), on='hole')
print('Is the Course Hole information correct?')
print(hole_info['course_name'][0])
hole_info.drop('course_name', axis=1, inplace=True)
# double checking date entry
hole_info

If the above infromation is correct please run the cell below

In [ ]:
golf_functions.load_sql(hole_info, 'Courses', 'hole_info')